In [1]:
import re
import pymysql

import numpy as np
import pandas as pd

In [ ]:
sel = """
SELECT  A2.TimerStr '时间',
        -- 日产量,
        产品规格,
        累计进砖,
        U46_实际温度,
        U47_实际温度,
        U48_实际温度,
        U49_实际温度,
        U50_实际温度
FROM (
    SELECT  TimerStr,
            YEAR(TimerStr) TimerYear,
            DAYOFYEAR(TimerStr) TimerDays,
            JProductNum '日产量',
            JProductSize '产品规格'
    FROM jproductnum_day 
) A1
RIGHT JOIN (
    SELECT  TimerInt,
            TimerStr,
            YEAR(TimerStr) TimerYear,
            DAYOFYEAR(TimerStr) TimerDays,
            POSVALUE_4078 '累计进砖'
    FROM j_cn_usertwp_历史数据_流量计
) A2 ON A2.TimerYear = A1.TimerYear
    AND A2.TimerDays = A1.TimerDays
LEFT JOIN (
    SELECT  TimerInt,
            POSVALUE_0918 'U46_实际温度',
            POSVALUE_0930 'U47_实际温度',
            POSVALUE_0942 'U48_实际温度',
            POSVALUE_0954 'U49_实际温度',
            POSVALUE_0966 'U50_实际温度'
    FROM j_cn_usertwp_温控表_实际值
) A3 ON A3.TimerInt = A2.TimerInt
"""

In [ ]:
sel = """
SELECT  TimerStr '时间',
        JProductNum '班产量'
    FROM jproductnum_shifttime 
"""

In [4]:
sel = """
    SELECT  timerstr '时间',
            POSVALUE_0463 'U34_实际温度',
            POSVALUE_0470 'U35_实际温度',
            POSVALUE_0477 'U36_实际温度',
            POSVALUE_0484 'U37_实际温度',
            POSVALUE_0491 'U38_实际温度'
    FROM j_cn_usertwp_温控表_实际值
    -- WHERE DATE(timerstr) >= '2020-10-29' AND DATE(timerstr) <= '2020-11-26'
"""

In [ ]:
# F20000A20
with pymysql.connect(
    host="10.10.141.126", database="jtemdata",
    user="root", password="",
) as conn:
    with conn.cursor(cursor=pymysql.cursors.DictCursor) as cur:
        cur.execute(sel)
        df = pd.DataFrame.from_dict(cur.fetchall())
df

In [6]:
# F15000A20
with pymysql.connect(
    host="10.0.3.45", database="jtemdata",
    user="root", password="",
) as conn:
    with conn.cursor(cursor=pymysql.cursors.DictCursor) as cur:
        cur.execute(sel)
        df = pd.DataFrame.from_dict(cur.fetchall())
df

,时间,U34_实际温度,U35_实际温度,U36_实际温度,U37_实际温度,U38_实际温度
0,2020-11-26 09:19:15,1215001,1215001,1190001,1153001,1151001
1,2020-11-26 09:20:15,1215001,1215001,1190001,1153001,1151001
2,2020-11-26 09:21:15,1215001,1215001,1190001,1153001,1151001
3,2020-11-26 09:22:15,1215001,1215001,1190001,1153001,1151001
4,2020-11-26 09:23:15,1215001,1215001,1190001,1154001,1151001
...,...,...,...,...,...,...
876,2020-11-26 23:55:20,1215001,1215001,1190001,1154001,1152001
877,2020-11-26 23:56:20,1215001,1215001,1190001,1154001,1152001
878,2020-11-26 23:57:20,1215001,1215001,1190001,1153001,1152001
879,2020-11-26 23:58:20,1214001,1214001,1190001,1153001,1152001


In [ ]:
# format time
df["时间"] = pd.to_datetime(df["时间"])
df = df.set_index("时间",drop=True)

df

In [ ]:
# Format product specification
size = df["产品规格"].str.split(
    re.compile(r"产品规格\||[×|]|\(\w+/\w+\)[1-9]\d*.\d+"),
    regex=True,expand=True
).dropna(axis=0).replace('',np.nan).dropna(axis=1).rename(
    columns={1:"长", 2:"宽", 3:"件/排"}
)

size

In [ ]:
# format value
df = (df.apply(pd.to_numeric,errors="ignore")-1)/1000

# df = (df.drop("产品规格",axis=1).apply(pd.to_numeric,errors="ignore")-1)/1000
# size = size.apply(pd.to_numeric,errors="ignore")
# df = pd.concat([df,size],axis=1)

df

In [ ]:
# Calculate output
df["产量"] = (df["长"]/1000) * (df["宽"]/1000) * df["件/排"] \
            * (df["累计进砖"]-df["累计进砖"].shift(1)).fillna(0)

df

In [ ]:
# Summarize every period of time
df = df.resample('1D').mean()

# df = pd.concat([
#         df["产量"].resample('1D').sum(),
#         df.filter(regex=r"[长宽件累U]").resample('1D').mean()
#     ],axis=1
# ).fillna(method='pad')

df

In [ ]:
# Original value
df.round(3).to_csv(
    'data-src-day.csv',
    date_format = '%Y-%m-%d',
    encoding = 'gbk'
)